In [1]:
import requests
from bs4 import BeautifulSoup
import json

# scraping a route page

In [2]:
route = "https://www.mountainproject.com/route/105933562/"

In [3]:
text = requests.get(route).text
soup = BeautifulSoup(text, 'html.parser')

In [4]:
data = json.loads("".join(soup.find("script", {"type":"application/ld+json"}).contents))
data

{'@context': 'http://schema.org/',
 '@type': 'Place',
 'name': 'Exum Ridge',
 'description': '5.5 Trad, Alpine',
 'image': 'https://cdn2.apstatic.com/photos/climb/106068473_medium_1558470536.jpg',
 'geo': {'@type': 'GeoCoordinates',
  'latitude': '43.74091933',
  'longitude': '-110.80252804'},
 'aggregateRating': {'@type': 'AggregateRating',
  'ratingValue': '3.7',
  'reviewCount': '789'}}

# scraping a user page

In [5]:
user = 'https://www.mountainproject.com/user/10959/frances-fierst'

In [6]:
links = []
text = requests.get(user + '/ticks').text
soup = BeautifulSoup(text, 'html.parser')
num_pages = int(soup.find_all('a', {"class":"no-click"})[2].contents[0].strip()[-1])

In [7]:
for i in range(num_pages):
    text = requests.get(user + '/ticks?page=' + str(i + 1)).text
    soup = BeautifulSoup(text, 'html.parser')
    all_links = soup.find_all('a')
    for link in all_links:
        if len(link.find_all('strong')) > 0 and len(link) < 2:
            links.append({link.find('strong').contents[0]: link.get('href')})
links

[{'Sleepy Hallow': 'https://www.mountainproject.com/route/106599007/sleepy-hallow'},
 {'Girls with Chim(n)ay': 'https://www.mountainproject.com/route/111170331/girls-with-chimnay'},
 {'Bushwhacker': 'https://www.mountainproject.com/route/106599016/bushwhacker'},
 {'Route 66': 'https://www.mountainproject.com/route/106517711/route-66'},
 {'Good Dobby': 'https://www.mountainproject.com/route/108416293/good-dobby'},
 {'You Break It You Buy It': 'https://www.mountainproject.com/route/109619465/you-break-it-you-buy-it'},
 {"Kibbles 'n Bits": 'https://www.mountainproject.com/route/118229581/kibbles-n-bits'},
 {'Wherever I May Roam': 'https://www.mountainproject.com/route/105821077/wherever-i-may-roam'},
 {'Unknown crack': 'https://www.mountainproject.com/route/119654556/unknown-crack'},
 {'Fingers to hands': 'https://www.mountainproject.com/route/114460429/fingers-to-hands'},
 {'Ringlock crack': 'https://www.mountainproject.com/route/114459436/ringlock-crack'},
 {'The Dihedral': 'https://www